In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_pickle("full_dataframe_20210803_150443.pkl")
data.reset_index(inplace=True)
data.drop(['index','event_id'], inplace=True, axis=1)
print(data.shape)
data.head()

(7680, 531)


,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,OBJECT1_CR_R,...,OBJECT2_CORR_CTDOT_R_4,OBJECT2_CORR_CTDOT_T_4,OBJECT2_CORR_CTDOT_N_4,OBJECT2_CORR_CTDOT_RDOT_4,OBJECT2_CORR_CNDOT_R_4,OBJECT2_CORR_CNDOT_T_4,OBJECT2_CORR_CNDOT_N_4,OBJECT2_CORR_CNDOT_RDOT_4,OBJECT2_CORR_CNDOT_TDOT_4,COLLISSION_PROBABILITY_TARGET
0,18753.0,14347.0,-700.1,-5172.4,18012.1,14.4,-13791.4,-3957.2,-30.000000,22.474480,...,-0.999926,0.378319,-0.845081,-0.301615,0.901438,-0.372185,0.971804,0.258500,-0.901937,-30.000000
1,23982.0,13574.0,22.3,10104.3,21749.5,-39.7,-12310.9,5718.9,-30.000000,78.188040,...,-0.999511,-0.675979,-0.776708,0.667410,0.911423,0.652982,0.860347,-0.641016,-0.911900,-30.000000
2,23709.0,12093.0,188.2,-13921.3,19191.2,19.7,-9788.5,-7101.8,-30.000000,31.349310,...,-0.999995,-0.973987,0.792263,0.961697,-0.517752,-0.543370,0.926369,0.531127,0.517949,-27.650917
3,314.0,2001.0,-18.4,-311.4,-42.7,0.5,-268.6,1983.8,-16.843451,24.527640,...,-0.999942,-0.121330,-0.007675,0.161742,0.087987,0.034350,0.360326,-0.070074,-0.088322,-30.000000
4,10978.0,10027.0,132.6,-8226.7,-7268.6,8.8,-6638.6,7515.2,-30.000000,1.362408,...,-0.999340,0.352616,-0.301764,-0.332939,0.229495,-0.077586,0.700588,0.085000,-0.229904,-30.000000


In [3]:
data=data.head(100)

In [4]:
data.shape

(100, 531)

In [5]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [6]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 75 x 531
Test dataframe dimension 25 x 531


In [7]:
Y_train = train["COLLISSION_PROBABILITY_TARGET"]
X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
Y_test = test["COLLISSION_PROBABILITY_TARGET"]
X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [8]:
X = X_train
y = Y_train

In [9]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2_score(preds, dtrain):
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_mean_absolute_error(preds, dtrain):
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2_score(preds, dtrain):
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True
            
      # Select metric
      metric='lgb_adjusted_r2_score'
      metric_feval=lgb_adjusted_r2_score

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (80, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 25)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=15, random_state=77, seed = 101,num_iterations=100)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.02011  |  0.9757   |  0.6138   |  0.03793  |  8.115    |  11.31    |  0.07901  |  93.05    |
|  2        | -0.02189  |  0.8623   |  0.2922   |  0.02773  |  10.2     |  20.73    |  0.08383  |  103.5    |
|  3        | -0.02008  |  0.7888   |  0.3248   |  0.03557  |  10.38    |  10.86    |  0.07496  |  98.09    |
|  4        | -0.02348  |  0.7527   |  0.1395   |  0.01533  |  7.534    |  21.27    |  0.007313 |  97.28    |
|  5        | -0.02019  |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.54    |  0.08247  |  90.93    |
|  6        | -0.02152  |  0.8714   |  0.4619   |  0.009418 |  7.446    |  10.23    |  0.05503  |  119.8    |
|  7        | -0.02002  |  0.8732   |  0.5574   |  0.02964  |  14.99    |  10.23    |  0.05953  |  80.43    |
|  8      

In [10]:
''' #with other optimization metric: MAE
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      def lgb_mean_absolute_error(preds, dtrain):
            labels = dtrain.get_label()
            return 'MAE', mean_absolute_error(labels, preds), True
      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':'lgb_mean_absolute_error'} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=lgb_mean_absolute_error)
              #print(cv_results)
              return np.max(cv_results['MAE-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (80, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 25)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=300, random_state=77, seed = 101,num_iterations=200) '''

' #with other optimization metric: MAE\ndef bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):\n      dtrain = lgb.Dataset(data=X, label=y)\n      def lgb_mean_absolute_error(preds, dtrain):\n            labels = dtrain.get_label()\n            return \'MAE\', mean_absolute_error(labels, preds), True\n      # Objective Function\n      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):\n              params = {      \'application\':\'regression\',\n                              \'num_iterations\': num_iterations,\n                              \'early_stopping_round\':50,\n                              \'verbose\':-1,\n                              \'metric\':\'lgb_mean_absolute_error\'} # Default parameters\n              params["num_leaves"] = int(round(num_leaves))\n              params["learning_rate"] = learning_rate\n              params[\'feature_fraction\'] = m

In [11]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

{'bagging_fraction': 0.7381976080829555,
 'feature_fraction': 0.6089408104979862,
 'learning_rate': 0.029524490037090158,
 'max_depth': 14.928056267581557,
 'min_child_weight': 10.606487236132702,
 'min_split_gain': 0.03259774136545973,
 'num_leaves': 119.04922083909936}

In [12]:
filename="opt_parameters_{}.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
a_file = open(filename, "wb")

pickle.dump(opt_parameters, a_file)

a_file.close()

In [13]:
a_file = open(filename,"rb")
output = pickle.load(a_file)
output

{'bagging_fraction': 0.7381976080829555,
 'feature_fraction': 0.6089408104979862,
 'learning_rate': 0.029524490037090158,
 'max_depth': 14.928056267581557,
 'min_child_weight': 10.606487236132702,
 'min_split_gain': 0.03259774136545973,
 'num_leaves': 119.04922083909936}

In [14]:
#optimizer.max['params']

In [15]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [16]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 500,
    "n_estimators": 10,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [17]:
#Construct a gradient boosting model.
gbm = lgb.LGBMRegressor(**hyper_params)

In [18]:
lgbm_train = lgb.Dataset(X, label=y)
lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [19]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                valid_sets=lgbm_eval,
                verbose_eval=20,
                #eval_metric='lgb_r2_score',
                early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 50.3436


In [20]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


" gbm.fit(X, y,\n        eval_set=[(X_test, Y_test)],\n        eval_metric='l1',\n        early_stopping_rounds=50) "

In [21]:
Y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration)

In [22]:
print('The r2 of prediction is:', r2_score(y, Y_pred))
print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

The r2 of prediction is: 0.19601226582362552
The MSE of prediction is: 37.5269483373594
The RMSE of prediction is: 6.125924284331255


In [23]:
aux_y=pd.DataFrame(y)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result.columns=["y_true","y_predicted"]
result

,y_true,y_predicted
0,-30.0,-27.631225
1,-30.0,-27.607603
2,-30.0,-27.620931
3,-30.0,-27.625700
4,-30.0,-27.549668
...,...,...
70,-30.0,-27.661609
71,-30.0,-27.626457
72,-30.0,-26.603934
73,-30.0,-27.621954


In [24]:
result.y_predicted.idxmax()

12

In [25]:
10**result.y_predicted[result.y_predicted.idxmax()]

1.0267633945005606e-25

In [26]:
10**result.y_true[result.y_predicted.idxmax()]

4.773000000000008e-13

In [27]:
10**result.y_true.max()

1.0200000000000009e-07

In [28]:
10**result.y_predicted[result.y_true.idxmax()]

1.0267633945005606e-25

In [29]:
result.y_true.idxmax()

33

In [30]:
10**result.y_predicted[4711]

KeyError: 4711